# SVR model

## Import libraries and download the data

In [3]:
#Download libraries
import re
#from collections import defaultdict
import pandas as pd
#import altair as alt
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
#from sklearn.svm import SVR
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import OneHotEncoder
#from preprocessing import preprocessing_na, clean_categorical
#import time
from drop import drop_columns
from PCA_data import *
from imputer import *
from drop import *

In [4]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1804425,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,90.0,244.20,0.157475,0.009783,0.147692
1,1812706,2,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
2,1812706,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
3,1812706,11,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,24.0,258.95,0.157475,0.009783,0.147692
4,1812706,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,258.95,0.157475,0.009783,0.147692


In [ ]:
imputer = create_imputer(df)
df1, df2 = impute_data(df, df)
clean_df = drop_columns(df1)

In [4]:
is_null = clean_df.isnull().sum()

In [5]:
is_null[is_null>0]

monthly_weekday_counts          45322
monthly_survey                  45322
name                            48762
total_events_across_state       48762
positive_events_across_state    48762
                                ...  
historic_revisit                32260
historic_safety                 30320
historic_travel                 32640
historic_variety                32940
unacast_session_count             617
Length: 66, dtype: int64

In [3]:
imputer = create_imputer(df)
df1, df2 = impute_data(df, df)
clean_data = get_pca_trans_data(df1.drop(columns=['external_id','monthly_weekday_counts','monthly_survey',
 'name',
 'date',
 'city',
 'state',
 'country',
 'closest_place_category',
 'income_class',
 'density_class',
 'climate',
 'county']), 0.95)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [11]:
categorical_features = df1.loc[:, df1.dtypes == "object"]
print(categorical_features.shape)
data = df1.copy()
numerical_data = data.drop(columns=list(categorical_features.columns))

(50120, 861)


In [22]:
df1_bis = df1.infer_objects()

In [23]:
df1_bis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50120 entries, 0 to 50119
Columns: 861 entries, external_id to local_amount_per_capita
dtypes: float64(421), int64(427), object(13)
memory usage: 329.2+ MB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50120 entries, 0 to 50119
Columns: 861 entries, external_id to local_amount_per_capita
dtypes: float64(393), int64(455), object(13)
memory usage: 329.2+ MB


In [26]:
list(df.loc[:, df.dtypes == "object"].columns)

['external_id',
 'monthly_weekday_counts',
 'monthly_survey',
 'name',
 'date',
 'city',
 'state',
 'country',
 'closest_place_category',
 'income_class',
 'density_class',
 'climate',
 'county']

# SVR Model

## Without categorical values

## With raw data

I am waiting for the preprocessing to be done to run this part

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(clean_data.drop(columns=['unacast_session_count', 'state', 'external_id']),
                                                    clean_data[['unacast_session_count']],
                                                    test_size=0.2,
                                                    random_state=2020)

In [ ]:
#Fit basic SVR with default values
t0 = time.time()
svr = SVR()
svr.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [ ]:
#Find the train and validation score
print(svr.score(X_train, y_train.to_numpy().ravel())
svr.score(X_valid, y_valid.to_numpy().ravel())

In [ ]:
(t1 - t0)/60

In [ ]:
print(svr.score(X_train, y_train.to_numpy().ravel()))
#Took roughly 1h30

SVR takes a long time to run, and gave really poor results. We will keep it aside for now, and see latter if we want to use it or not.

I kept the following code for when we will have more computational power to run SVR.

In [ ]:
"""
#Kernel = ‘linear’
t0 = time.time()
parameters = {'C':np.logspace(-5,5,40), 'gamma':np.logspace(-10,10,40)}
svr = SVR(kernel='linear')
svr_opt = RandomizedSearchCV(svr, parameters, cv=5, iid=False, n_iter=100)
svr_opt.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()
print('The best parameter combination is {}.'.format(svc_opt.best_params_))
print('The accuracy on the test split is {:.2f}.'.format(svc_opt.score(X_test,y_test)))
total = t1-t0
print(total)
"""

## Catboost

### Raw data

In [ ]:
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [ ]:
(t1-t0)/60

In [ ]:
# Get R2 score
model.score(X_train, y_train.to_numpy().ravel())

In [ ]:
model.score(X_valid, y_valid.to_numpy().ravel())

### Normalized data

In [ ]:
#Drop categorical features
categorical_features = X_train.loc[:, clean_data.dtypes == "object"]
categorical_features.columns

In [ ]:
#Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.drop(columns = list(categorical_features.columns)))
X_valid_scaled = scaler.transform(X_valid.drop(columns = list(categorical_features.columns)))

In [ ]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train_scaled, y_train.to_numpy().ravel())
t1 = time.time()

In [ ]:
(t1-t0)/60

In [ ]:
# Get R2 score
model.score(X_train_scaled, y_train.to_numpy().ravel())

In [ ]:
model.score(X_valid_scaled, y_valid.to_numpy().ravel())

### Log transformed response

In [ ]:
y_train_log = np.log(np.where(y_train==0, 1, y_train))
y_valid_log = np.log(np.where(y_valid==0, 1, y_valid))

In [ ]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train, y_train_log.ravel())
t1 = time.time()

In [ ]:
(t1-t0)/60

In [ ]:
# Get R2 score
model.score(X_train, y_train_log.ravel())

In [ ]:
model.score(X_valid, y_valid_log.ravel())

## Hyperparameters tuning : Random search (Waiting for the EC2 to run the rest)

### Normalized data

In [ ]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train_scaled, y_train.to_numpy().ravel())

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid_scaled,y_valid)))

### Log transformation of the response

In [ ]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train, y_train_log)

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid,y_valid_log)))

### Log transformation of the response + Normalization

In [ ]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train_scaled, y_train_log)

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid_scaled,y_valid_log)))